# API Behaviour Analysis Towards Improving Headways Algorithm

In [1]:
#First we import the requested modules
import pandas as pd
from pandas.io.json import json_normalize
import json

import math
import numpy as np

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
pio.templates.default = 'plotly_white'
pd.set_option("display.precision", 3)

import datetime
from datetime import timedelta

from pandarallel import pandarallel
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
pandarallel.initialize()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [18]:
#Available colors
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [19]:
f = '../Data/'

## Data belonging to last week

In [20]:
#Read week df
week_df = pd.read_csv(f+'RealTime/buses_data_week_cleaned.csv',
    dtype={
        'line': 'uint16',
        'direction': 'uint16',
        'stop': 'str',
        'bus': 'str',
        'estimateArrive': 'uint16'
    }
)[['line','direction','stop','bus','datetime','estimateArrive']]

#Parse the dates
week_df['datetime'] = pd.to_datetime(week_df['datetime'], errors = 'coerce', format='%Y-%m-%dT%H:%M:%S.%f')

In [21]:
week_df.head()

,line,direction,stop,bus,datetime,estimateArrive
0,18,1,490015040W,BF67GMX,2021-01-22 09:05:23.221955600,1373
1,18,1,490015040W,BF67GLY,2021-01-22 09:05:23.221955600,1205
2,18,1,490015040W,BF67GKZ,2021-01-22 09:05:23.221955600,1615
3,18,1,490015040W,BF67GMY,2021-01-22 09:05:23.221955600,94
4,18,1,490015040W,BF67GMY,2021-01-22 09:05:23.221955600,94


In [22]:
week_df.describe()

,line,direction,estimateArrive
count,443461.000,443461.000,443461.000
mean,20.841,1.503,886.927
std,3.437,0.500,517.574
min,18.000,1.000,1.000
25%,18.000,1.000,439.000
50%,18.000,2.000,881.000
75%,25.000,2.000,1333.000
max,25.000,2.000,1829.000


In [23]:
week_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443461 entries, 0 to 443460
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   line            443461 non-null  uint16        
 1   direction       443461 non-null  uint16        
 2   stop            443461 non-null  object        
 3   bus             443461 non-null  object        
 4   datetime        443461 non-null  datetime64[ns]
 5   estimateArrive  443461 non-null  uint16        
dtypes: datetime64[ns](1), object(2), uint16(3)
memory usage: 12.7+ MB


In [24]:
week_df.datetime.dt.date.value_counts()

2021-01-22    443461
Name: datetime, dtype: int64

In [25]:
#Day types
day_type_dict = { #0 = Monday, 1 = Tuesday ...
    'LA' : [0,1,2,3,4], #LABORABLES
    'LJ' : [0,1,2,3], #LUNES A JUEVES
    'VV' : [4], #VIERNES
    'SA' : [5], #SABADOS
    'FE' : [6], #DOMIGOS O FESTIVOS
}

In [26]:
#Load line_stops_dict
with open(f+'Static/lines_dict.json', 'r') as file:
    lines_dict = json.load(file)

In [57]:
#Load times between stops data
times_bt_stops = pd.read_csv(f+'Processed/times_bt_stops.csv',
    dtype={
        'line': 'uint16',
        'direction': 'uint16',
        'st_hour': 'uint16',
        'end_hour': 'uint16',
        'stopA': 'str',
        'stopB': 'str',
        'bus': 'str',
        'trip_time':'float16',
        'api_trip_time':'int16'
    }
)
#Parse the dates
times_bt_stops['date'] = pd.to_datetime(times_bt_stops['date'], format='%Y-%m-%d')

## Headways Processing

In [58]:
lines_sel = ['18']

In [59]:
def get_headways(df,lines_sel) :
    '''
    Returns a dataframe with the headways between buses for every day, line and hour range.

    Parameters
    -----------------
        df: Dataframe
            Data to process
    '''
    #For every line collected
    lines = lines_sel

    #Get new dictionaries and process the lines
    dfs_list = []
    for line in lines :
        line_df = df.loc[df.line == int(line)]
        dates = line_df.datetime.dt.date.unique().tolist()
        #dfs = (Parallel(n_jobs=num_cores,max_nbytes=None)(delayed(process_day_df)(line_df,date) for date in dates))
        dfs = [process_day_df(line_df, dates[0])]
        dfs_list += dfs

    #Concatenate dataframes
    processed_df = pd.concat(dfs_list).sort_values(by=['line','datetime','direction'], ascending=True).reset_index(drop = True)
    return processed_df

In [60]:
#FUNCTIONS
def process_day_df(line_df,date) :
    '''
    Returns the dataframe with the headways between buses for the line and day selected.

    Parameters
    -----------------
        line_df: Dataframe
        year,month,day : ints
    '''

    #Select day data
    day_df = line_df.loc[line_df.datetime.dt.date == date].sort_values(by='datetime',ascending=True)

    # If the dataframe is empty
    if day_df.shape[0] == 0 :
        return pd.DataFrame([])

    #Line id
    line = day_df.iloc[0].line
    
    #Day type
    now = day_df.datetime.iloc[0]
    if (now.weekday() >= 0) and (now.weekday() <= 4) :
        day_type = 'LA'
    elif now.weekday() == 5 :
        day_type = 'SA'
    else :
        day_type = 'FE'

    #Stops of each line reversed
    stops1 = lines_dict[str(line)]['1']['stops'][-3:3:-1]
    stops2 = lines_dict[str(line)]['2']['stops'][-3:3:-1]

    #Appearance order buses list
    ap_order_dir1,ap_order_dir2 = [],[]
    bus_ttls1,bus_ttls2 = {},{}
    bus_cons_disap1,bus_cons_disap2 = {},{}
    bus_cons_ap1,bus_cons_ap2 = {},{}

    rows_list = []
    if day_df.shape[0] > 0 :
        #First interval for the iteration :
        actual_time = day_df.iloc[0].datetime
        start_interval = actual_time - timedelta(seconds=10)
        end_interval = actual_time + timedelta(seconds=15)

        #Iterate over bursts
        last_hour = -10
        while True :
            int_df = day_df.loc[(day_df.datetime > start_interval) & (day_df.datetime < end_interval)]

            #Drop duplicate buses keeping the one with lowest estimateArrive
            int_df = int_df.sort_values('estimateArrive').drop_duplicates('bus', keep='first')

            if actual_time.hour > last_hour :
                last_hour = actual_time.hour
                #Process mean times between stops
                tims_bt_stops = times_bt_stops.loc[(times_bt_stops.line == line) & \
                                                    (times_bt_stops.date.dt.weekday.isin(day_type_dict[day_type])) & \
                                                    (times_bt_stops.st_hour >= last_hour) & \
                                                    (times_bt_stops.st_hour < last_hour+2)]
                #Group and get the mean values
                tims_bt_stops = tims_bt_stops.groupby(['line','direction','stopA','stopB']).mean()
                tims_bt_stops = tims_bt_stops.reset_index()[['line','direction','stopA','stopB','trip_time','api_trip_time']]

            if int_df.shape[0] > 0 :
                #All stops of the line
                stops = stops1 + stops2
                stop_df_list = []
                buses_out1,buses_out2 = [],[]
                direction = 1
                for i in range(len(stops)) :
                    stop = stops[i]
                    if i == 0 :
                        mean_time_to_stop = 0
                    elif i == len(stops1) :
                        mean_time_to_stop_1 = mean_time_to_stop
                        mean_time_to_stop = 0
                        direction = 2
                    else :
                        mean_df = tims_bt_stops.loc[(tims_bt_stops.stopA == stop) & \
                                        (tims_bt_stops.direction == direction)]
                        if mean_df.shape[0] > 0 :
                            mean_time_to_stop += mean_df.iloc[0].trip_time
                        else :
                            continue

                    stop_df = int_df.loc[(int_df.stop == stop) & \
                                        (int_df.direction == direction)]

                    #Drop duplicates, recalculate estimateArrive and append to list
                    stop_df = stop_df.drop_duplicates('bus',keep='first')

                    if (stop == stops1[-1]) or (stop == stops2[-1]) :
                        if direction == 1 :
                            buses_out1 += stop_df.bus.unique().tolist()
                        else :
                            buses_out2 += stop_df.bus.unique().tolist()
                    elif (stop == stops1[0]) or (stop == stops2[0]) :
                        buses_near = stop_df.loc[stop_df.estimateArrive < 60]
                        if buses_near.shape[0] > 0 :
                            if direction == 1 :
                                buses_out1 += buses_near.bus.unique().tolist()
                            else :
                                buses_out2 += buses_near.bus.unique().tolist()
                        
                    stop_df.estimateArrive = stop_df.estimateArrive + mean_time_to_stop
                    stop_df_list.append(stop_df)

                mean_time_to_stop_2 = mean_time_to_stop

                #Concatenate and group them
                stops_df = pd.concat(stop_df_list)

                #Eliminate TTLS longer than mean time to go through hole line direction
                stops_df = stops_df[(stops_df.direction == 1) & (stops_df.estimateArrive < mean_time_to_stop_1) | \
                                    (stops_df.direction == 2) & (stops_df.estimateArrive < mean_time_to_stop_2)]

                #Group by bus and direction
                #stops_df = stops_df.groupby(['bus','direction']).mean().sort_values(by=['estimateArrive'])
                stops_df = stops_df.sort_values(by=['estimateArrive'])
                stops_df = stops_df.reset_index().drop_duplicates('bus',keep='first')
                #Loc buses not given by first stop
                stops_df = stops_df.loc[((stops_df.direction == 1) & (~stops_df.bus.isin(buses_out1))) | \
                                        ((stops_df.direction == 2) & (~stops_df.bus.isin(buses_out2))) ]

                #Update appearance order lists
                if (ap_order_dir1 == []) & (ap_order_dir2 == []) :
                    TH = 0
                else : 
                    TH = 5

                #Set min difference bt last TH TTLSs of a bus to consider it is moving
                MIN_TTLS_DIFF = 5*TH

                stops_df_dest1 = stops_df[stops_df.direction == 1].sort_values(by=['estimateArrive'])
                if stops_df_dest1.shape[0] > 0 :  
                    buses_dest1 = stops_df_dest1.bus.tolist()
                    etas_dest1 = stops_df_dest1.estimateArrive.tolist()
                    for i in range(len(buses_dest1)):
                        if buses_dest1[i] not in bus_ttls1:
                            bus_ttls1[buses_dest1[i]] = []
                            bus_ttls1[buses_dest1[i]].append(etas_dest1[i])
                        elif len(bus_ttls1[buses_dest1[i]]) < TH : 
                            bus_ttls1[buses_dest1[i]].append(etas_dest1[i])
                        else : 
                            bus_ttls1[buses_dest1[i]].pop(0)
                            bus_ttls1[buses_dest1[i]].append(etas_dest1[i])

                        ttls_diff = max(bus_ttls1[buses_dest1[i]]) - min(bus_ttls1[buses_dest1[i]])

                        if buses_dest1[i] not in bus_cons_ap1 :
                            bus_cons_ap1[buses_dest1[i]] = 0
                        if  buses_dest1[i] not in bus_cons_disap1 :
                            bus_cons_disap1[buses_dest1[i]] = 0

                        if (bus_cons_ap1[buses_dest1[i]] > TH) :
                            if (buses_dest1[i] not in ap_order_dir1) : 
                                #Append to apearance list
                                if i > 0 : 
                                    bus_bef = buses_dest1[i-1]
                                    for k in range(len(ap_order_dir1)) : 
                                        if ap_order_dir1[k] == bus_bef : 
                                            ap_order_dir1.insert(k+1,buses_dest1[i])
                                else : 
                                    ap_order_dir1.insert(0,buses_dest1[i])
                    

                    #Update times without appering
                    for bus in bus_cons_disap1 :
                        if bus not in buses_dest1 :
                            bus_cons_disap1[bus] += 1
                            bus_cons_ap1[bus] = 0
                            if bus_cons_disap1[bus] > TH :
                                if bus in ap_order_dir1 :
                                    ap_order_dir1.remove(bus)
                        else :
                            bus_cons_disap1[bus] = 0
                            bus_cons_ap1[bus] += 1
                
                stops_df_dest2 = stops_df[stops_df.direction == 2].sort_values(by=['estimateArrive'])
                if stops_df_dest2.shape[0] > 0 :
                    buses_dest2 = stops_df_dest2.bus.tolist()
                    etas_dest2 = stops_df_dest2.estimateArrive.tolist()
                    for i in range(len(buses_dest2)):
                        if buses_dest2[i] not in bus_ttls2:
                            bus_ttls2[buses_dest2[i]] = []
                            bus_ttls2[buses_dest2[i]].append(etas_dest2[i])
                        elif len(bus_ttls2[buses_dest2[i]]) < TH : 
                            bus_ttls2[buses_dest2[i]].append(etas_dest2[i])
                        else : 
                            bus_ttls2[buses_dest2[i]].pop(0)
                            bus_ttls2[buses_dest2[i]].append(etas_dest2[i])

                        ttls_diff = max(bus_ttls2[buses_dest2[i]]) - min(bus_ttls2[buses_dest2[i]])

                        if buses_dest2[i] not in bus_cons_ap2 :
                            bus_cons_ap2[buses_dest2[i]] = 0
                        if  buses_dest2[i] not in bus_cons_disap2 :
                            bus_cons_disap2[buses_dest2[i]] = 0

                        if (bus_cons_ap2[buses_dest2[i]] > TH) :
                            if (buses_dest2[i] not in ap_order_dir2) : 
                                #Append to apearance list
                                if i > 0 : 
                                    bus_bef = buses_dest2[i-1]
                                    for k in range(len(ap_order_dir2)) : 
                                        if ap_order_dir2[k] == bus_bef : 
                                            ap_order_dir2.insert(k+1,buses_dest2[i])
                                            break
                                else : 
                                    ap_order_dir2.insert(0,buses_dest2[i])

                    #Update times without appering
                    for bus in bus_cons_disap2 :
                        if bus not in buses_dest2 :
                            bus_cons_disap2[bus] += 1
                            bus_cons_ap2[bus] = 0
                            if bus_cons_disap2[bus] > TH :
                                if bus in ap_order_dir2 :
                                    ap_order_dir2.remove(bus)
                        else :
                            bus_cons_disap2[bus] = 0
                            bus_cons_ap2[bus] += 1

                #Reorder df according to appearance list
                rows,last_ttls1,last_bus1,last_ttls2,last_bus2 = [],0,0,0,0
                for bus in ap_order_dir1 :
                    bus_df = stops_df_dest1[stops_df_dest1.bus == bus]
                    if bus_df.shape[0] > 0 :
                        if (last_ttls1 > bus_df.iloc[0].estimateArrive) & (TH+1 < bus_cons_ap1[bus] < TH+4) :
                            rows.pop(-1)
                            rows.append(bus_df.iloc[0])

                            ap_order_dir1.remove(last_bus1)
                            bus_cons_ap1[last_bus1] = TH + 1
                            bus_cons_disap1[last_bus1] = TH + 1
                            break
                        else :
                            rows.append(bus_df.iloc[0])
                            last_ttls1 = bus_df.iloc[0].estimateArrive
                            last_bus1 = bus
                    
                for bus in ap_order_dir2 :
                    bus_df = stops_df_dest2[stops_df_dest2.bus == bus]
                    if bus_df.shape[0] > 0 :
                        if (last_ttls2 > bus_df.iloc[0].estimateArrive) & (TH+1 < bus_cons_ap2[bus] < TH+4) :
                            rows.pop(-1)
                            rows.append(bus_df.iloc[0])
                            
                            ap_order_dir2.remove(last_bus2)
                            bus_cons_ap2[last_bus2] = TH + 1
                            bus_cons_disap2[last_bus2] = TH + 1
                            break
                        else :
                            rows.append(bus_df.iloc[0])
                            last_ttls2 = bus_df.iloc[0].estimateArrive
                            last_bus2 = bus

                stops_df = pd.DataFrame(rows)

                '''
                print('\n[LONDON] Line {} - Apearance Order Dict'.format(line))
                
                print('\nAp Order List')
                print(ap_order_dir1)
                print(ap_order_dir2)

                print('\nLast TTLSs')
                print(bus_ttls1)
                print(bus_ttls2)
                
                print('\nConsecutive Disapearances')
                print(bus_cons_disap1)
                print(bus_cons_disap2)

                print('\nConsecutive Apearances')
                print(bus_cons_ap1)
                print(bus_cons_ap2)
                '''

                #Calculate time intervals
                if stops_df.shape[0] > 0 :
                    hw_pos1 = 0
                    hw_pos2 = 0
                    for i in range(stops_df.shape[0]) :
                        est1 = stops_df.iloc[i]

                        direction = est1.direction
                        if ((direction == 1) & (hw_pos1 == 0)) or ((direction == 2) & (hw_pos2 == 0))  :
                            #Create dataframe row
                            row = {}
                            row['datetime'] = actual_time
                            row['line'] = line
                            row['direction'] = direction
                            row['busA'] = 0
                            row['busB'] = est1.bus
                            row['hw_pos'] = 0
                            row['headway'] = 0
                            row['busB_ttls'] = int(est1.estimateArrive)

                            #Append row to the list of rows
                            rows_list.append(row)

                            #Increment hw pos
                            if direction == 1 :
                                hw_pos1 += 1
                            else :
                                hw_pos2 += 1

                        if i < (stops_df.shape[0] - 1) :
                            est2 = stops_df.iloc[i+1]
                        else :
                            break

                        if est1.direction == est2.direction :
                            headway = int(est2.estimateArrive-est1.estimateArrive)

                            #Create dataframe row
                            row = {}
                            row['datetime'] = actual_time
                            row['line'] = line
                            row['direction'] = direction
                            row['busA'] = est1.bus
                            row['busB'] = est2.bus
                            row['hw_pos'] = hw_pos1 if direction == 1 else hw_pos2
                            row['headway'] = headway
                            row['busB_ttls'] = int(est2.estimateArrive)

                            #Append row to the list of rows
                            rows_list.append(row)

                            #Increment hw pos
                            if direction == 1 :
                                hw_pos1 += 1
                            else :
                                hw_pos2 += 1

            #Update iteration interval
            next_df = day_df.loc[(day_df.datetime > end_interval) & \
                                    (day_df.datetime < end_interval + timedelta(minutes=5))]
            if next_df.shape[0] == 0 :
                next_df = day_df.loc[(day_df.datetime > end_interval)]

            if next_df.shape[0] != 0 :
                actual_time = next_df.iloc[0].datetime
                start_interval = actual_time - timedelta(seconds=10)
                end_interval = actual_time + timedelta(seconds=15)
            else :
                break

    return pd.DataFrame(rows_list)

In [61]:
headways = get_headways(week_df, lines_sel)

In [62]:
headways.head(10)

,datetime,line,direction,busA,busB,hw_pos,headway,busB_ttls
0,2021-01-22 12:49:13.788023900,18,1,0,LJ16EWX,0,0,211
1,2021-01-22 12:49:13.788023900,18,1,LJ16EWX,LJ16EWZ,1,442,654
2,2021-01-22 12:49:13.788023900,18,1,LJ16EWZ,BF67GKV,2,57,711
3,2021-01-22 12:49:13.788023900,18,1,BF67GKV,BF67GME,3,22,733
4,2021-01-22 12:49:13.788023900,18,1,BF67GME,BF67GKO,4,369,1102
5,2021-01-22 12:49:13.788023900,18,1,BF67GKO,LJ15JYZ,5,745,1847
6,2021-01-22 12:49:13.788023900,18,1,LJ15JYZ,BF67GKU,6,226,2074
7,2021-01-22 12:49:13.788023900,18,1,BF67GKU,BF67GMO,7,126,2201
8,2021-01-22 12:49:13.788023900,18,1,BF67GMO,BF67GLJ,8,358,2559
9,2021-01-22 12:49:13.788023900,18,1,BF67GLJ,BF67GNN,9,286,2846


In [63]:
hws = headways[headways.hw_pos > 0]

In [64]:
hws_neg = headways[headways.headway < 0]

In [66]:
hws_neg.describe()

,line,direction,hw_pos,headway,busB_ttls
count,393.0,393.000,393.000,393.000,393.000
mean,18.0,1.565,3.150,-212.280,1247.628
std,0.0,0.496,1.841,163.744,806.853
min,18.0,1.000,1.000,-804.000,63.000
25%,18.0,1.000,1.000,-328.000,526.000
50%,18.0,2.000,3.000,-194.000,1176.000
75%,18.0,2.000,5.000,-75.000,1920.000
max,18.0,2.000,8.000,-1.000,3549.000


In [68]:
hws_neg.busA.value_counts()

BF67GMY    71
BF67GMV    61
BF67GLJ    46
BF67GMX    45
BF67GMU    33
BF67GKK    27
LJ15JZA    24
BF67GMO    14
BF67GKN    13
LJ16EWW    12
BF67GKU    12
BF67GKJ    11
LJ16EWZ     7
BF67GKV     6
BF67GLY     6
BF67GNN     3
BF67GMG     2
Name: busA, dtype: int64

In [70]:
hws_neg.busB.value_counts()

BF67GNN    107
BF67GMX     69
BF67GLY     45
BF67GLZ     33
LJ16EWX     27
LJ66TRZ     24
BF67GNJ     13
BF67GMO     12
BF67GMU     12
BF67GLJ     11
LJ15JZA     11
BF67GKO      6
LJ16EWV      6
BF67GME      5
BF67GMV      3
BF67GKJ      3
BF67GKK      2
BF67GKV      2
BF67GKE      2
Name: busB, dtype: int64

In [78]:
hws_neg[hws_neg.busA == 'BF67GMY']

,datetime,line,direction,busA,busB,hw_pos,headway,busB_ttls
4325,2021-01-22 14:32:04.156055800,18,1,BF67GMY,BF67GMX,8,-5,3103
4347,2021-01-22 14:32:34.317082400,18,1,BF67GMY,BF67GMX,8,-26,3033
4368,2021-01-22 14:33:04.106037300,18,1,BF67GMY,BF67GMX,7,-8,3022
4389,2021-01-22 14:33:34.245087400,18,1,BF67GMY,BF67GMX,6,-49,2909
4407,2021-01-22 14:34:04.942933200,18,1,BF67GMY,BF67GMX,6,-68,2934
...,...,...,...,...,...,...,...,...
5674,2021-01-22 15:05:32.498917800,18,1,BF67GMY,BF67GMX,1,-341,184
5695,2021-01-22 15:06:02.495099400,18,1,BF67GMY,BF67GMX,1,-371,124
5717,2021-01-22 15:06:27.312620700,18,1,BF67GMY,BF67GMX,1,-328,127
5739,2021-01-22 15:07:02.480217800,18,1,BF67GMY,BF67GKE,1,-244,171


In [79]:
week_df[week_df.bus == 'BF67GNN']

,line,direction,stop,bus,datetime,estimateArrive
11078,18,1,490005320E,BF67GNN,2021-01-22 12:49:06.630751700,1500
11112,18,1,490003444E,BF67GNN,2021-01-22 12:49:08.154080100,1270
11120,18,1,490005720E,BF67GNN,2021-01-22 12:49:08.247002800,80
11174,18,1,490008865E,BF67GNN,2021-01-22 12:49:08.822194900,38
11179,18,1,490005491E,BF67GNN,2021-01-22 12:49:08.840786900,557
...,...,...,...,...,...,...
443123,18,2,490015040Z,BF67GNN,2021-01-22 16:59:03.699411100,411
443137,18,2,490000252V,BF67GNN,2021-01-22 16:59:04.111648200,44
443177,18,2,490007666W,BF67GNN,2021-01-22 16:59:04.508539700,1498
443366,18,2,490012788W,BF67GNN,2021-01-22 16:59:07.464794100,1791


In [80]:
week_df[(week_df.bus == 'BF67GNN') & (week_df.stop == '490005320E')]

,line,direction,stop,bus,datetime,estimateArrive
11078,18,1,490005320E,BF67GNN,2021-01-22 12:49:06.630751700,1500
12308,18,1,490005320E,BF67GNN,2021-01-22 12:53:37.698889500,1229
13611,18,1,490005320E,BF67GNN,2021-01-22 12:56:11.174804200,1075
13621,18,1,490005320E,BF67GNN,2021-01-22 12:56:11.174804200,1075
14302,18,1,490005320E,BF67GNN,2021-01-22 12:56:37.866437000,1049
...,...,...,...,...,...,...
379635,18,1,490005320E,BF67GNN,2021-01-22 16:24:04.616727700,115
381616,18,1,490005320E,BF67GNN,2021-01-22 16:25:04.785935500,55
382647,18,1,490005320E,BF67GNN,2021-01-22 16:25:34.793988800,25
383578,18,1,490005320E,BF67GNN,2021-01-22 16:26:05.414420300,21
